In [ ]:
import torch

from transformers import BertTokenizer, BertModel, AutoTokenizer, DebertaModel, AutoModel, PreTrainedModel

def seed(seed_val):
    import random
    import numpy as np
    import torch
    # import tensorflow as tf
    # tf.random.set_seed(seed_value)
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
seed(42)

class deberta:

    def __init__(self):
        self.__name__ = 'microsoft/deberta-base'
        self.__num_node_features__ = 768 
        self.device = 'cpu'
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
# Load model directly
        self.model = AutoModel.from_pretrained("microsoft/deberta-base")
        # self.model = DebertaModel.from_pretrained("microsoft/deberta-base")
        
        # self.__output_dim__ = self.__model__.
    # @property
    def parameters(self):
        return self.model.parameters()

    @property
    def num_node_features(self):
        return 768

    def to(self, device):
        self.model = self.model.to(device)
        self.device = device
        return self

    def forward(self, text):

        def model_forward_input(input):
            input = self.tokenizer(input, return_tensors='pt').to(self.device)
            output = self.model(**input).last_hidden_state.mean(dim=1)
            # print(output.shape)
            # return self.model(**input).last_hidden_state.mean(dim=1)
            # print(output.shape)
            return torch.squeeze(output)

        return torch.stack(list(map(model_forward_input, text)))

    def __call__(self, data):
        if isinstance(data, str):
            return self.forward([data])
        if isinstance(data, list):
            return self.forward(data)

In [ ]:
from typing import Optional

import torch
from torch import Tensor


class History(torch.nn.Module):
    r"""A historical embedding storage module."""
    def __init__(self, num_embeddings: int, embedding_dim: int, device=None):
        super().__init__()

        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim

        pin_memory = device is None or str(device) == 'cpu'
        self.emb = torch.empty(num_embeddings, embedding_dim, device=device,
                               pin_memory=pin_memory)

        self._device = 'cpu'

        self.reset_parameters()

    def reset_parameters(self):
        self.emb.fill_(0)

    def _apply(self, fn):
        # Set the `_device` of the module without transfering `self.emb`.
        self._device = fn(torch.zeros(1)).device
        return self

    @torch.no_grad()
    def pull(self, n_id: Optional[Tensor] = None) -> Tensor:
        out = self.emb
        if n_id is not None:
            assert n_id.device == self.emb.device
            out = out.index_select(0, n_id)
        return out.to(device=self._device)

    @torch.no_grad()
    def push(self, x, n_id: Optional[Tensor] = None,
             offset: Optional[Tensor] = None, count: Optional[Tensor] = None):

        if n_id is None and x.size(0) != self.num_embeddings:
            raise ValueError

        elif n_id is None and x.size(0) == self.num_embeddings:
            self.emb.copy_(x)

        elif offset is None or count is None:
            assert n_id.device == self.emb.device
            self.emb[n_id] = x.to(self.emb.device).detach()

        else:  # Push in chunks:
            src_o = 0
            x = x.to(self.emb.device)
            for dst_o, c, in zip(offset.tolist(), count.tolist()):
                self.emb[dst_o:dst_o + c] = x[src_o:src_o + c]
                src_o += c

    def forward(self, *args, **kwargs):
        """"""
        raise NotImplementedError

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.num_embeddings}, '
                f'{self.embedding_dim}, emb_device={self.emb.device}, '
                f'device={self._device})')


In [ ]:
device = torch.device(0)

In [ ]:
lm = deberta().to(device)

In [ ]:
import dgl
import torch
import numpy as np
# from ogb.nodeproppred import DglNodePropPredDataset

from data_utils import load_data

# dataset = dgl.data.CoraGraphDataset()
# # device = 'cuda'      # change to 'cuda' for GPU
# graph = dataset[0]
graph, num_classes, text = load_data('ogbn-arxiv', use_dgl=True, use_text=True)
# print(graph.ndata)
# print(type(graph))
# graph = (dataset)


# get text feat
# dataset, num_classes, text = load_data('cora', use_dgl=True, use_text=True)
# features = []
# with torch.no_grad():
#     for i, t in enumerate(text):
#         if i % 1000 == 0:
#             print(i)
#         features.append(lm(t).squeeze().cpu())
# features = torch.stack(features)
# torch.save(features, 'arxiv.pt')

In [ ]:
import dgl
import torch
import numpy as np
from ogb.nodeproppred import DglNodePropPredDataset

from data_utils import load_data

# dataset = dgl.data.CoraGraphDataset()
# # device = 'cuda'      # change to 'cuda' for GPU
# graph = dataset[0]
# graph, num_classes, text = load_data('ogbn-arxiv', use_dgl=True, use_text=True)
# print(graph.ndata)
# print(type(graph))
# graph = (dataset)


# get text feat
# dataset, num_classes, text = load_data('cora', use_dgl=True, use_text=True)
# features = []
# with torch.no_grad():
#     for t in text:
#         features.append(lm(t).squeeze().cpu())
# features = torch.stack(features)
# torch.save(features, 'arxiv.pt')
# print("graph.ndata['feat'].shape", graph.ndata['feat'].shape)
# print("features.shape", features.shape)
features = torch.load('dataset/arxiv.pt')
graph.ndata['x'] = features

train_mask = graph.ndata['train_mask']
# train_mask = graph.train_mask
train_nids = torch.nonzero(train_mask, as_tuple=False).squeeze()
val_mask = graph.ndata['val_mask']
# val_mask = graph.val_mask
val_nids = torch.nonzero(val_mask, as_tuple=False).squeeze()
test_mask = graph.ndata['test_mask']
# test_mask = graph.test_mask
test_nids = torch.nonzero(test_mask, as_tuple=False).squeeze()

sampler = dgl.dataloading.NeighborSampler([4, 4])

train_dataloader = dgl.dataloading.DataLoader(
    # The following arguments are specific to DGL's DataLoader.
    graph,              # The graph
    train_nids,         # The node IDs to iterate over in minibatches
    sampler,            # The neighbor sampler
    device=device,      # Put the sampled MFGs on CPU or GPU
    # The following arguments are inherited from PyTorch DataLoader.
    batch_size=16,    # Batch size
    shuffle=True,       # Whether to shuffle the nodes for every epoch
    drop_last=True,    # Whether to drop the last incomplete batch
    num_workers=0       # Number of sampler processes
)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import tqdm
from dgl.nn import SAGEConv

class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.history = [History(graph.num_nodes(), embedding_dim=in_feats, device='cpu'), History(graph.num_nodes(), embedding_dim=h_feats, device='cpu')]
        self.conv1 = SAGEConv(in_feats, h_feats, aggregator_type='mean')
        self.conv2 = SAGEConv(h_feats, num_classes, aggregator_type='mean')
        self.h_feats = h_feats


    def push_and_pull(self, history: History, x: Tensor,
                      batch_size: Optional[int] = None,
                      n_id: Optional[Tensor] = None,
                      offset: Optional[Tensor] = None,
                      count: Optional[Tensor] = None) -> Tensor:
        history.push(x[:batch_size], n_id[:batch_size], offset, count)
        h = history.pull(n_id[batch_size:]).to(x.device)
        return torch.cat([x[:batch_size], h], dim=0)

    @torch.no_grad()
    def forward_once(self, mfgs, x):
        self.history[0].push(x, mfgs[0].srcdata['_ID'].cpu())
        h_dst = x[:mfgs[0].num_dst_nodes()]  # <---
        # print("h_dst.shape", h_dst.shape)
        h = self.conv1(mfgs[0], (x, h_dst))  # <---
        h = F.relu(h)
        self.history[1].push(h, mfgs[1].srcdata['_ID'].cpu())


    def forward(self, mfgs, x, batch_size):
        # Lines that are changed are marked with an arrow: "<---"
        # print("x.shape: ", x.shape)
        x = self.push_and_pull(self.history[0], x, batch_size, mfgs[0].srcdata['_ID'].cpu())
        # print("x.shape: ", x.shape)
        h_dst = x[:mfgs[0].num_dst_nodes()]  # <---
        # print("h_dst.shape", h_dst.shape)
        h = self.conv1(mfgs[0], (x, h_dst))  # <---
        h = F.relu(h)
        h = self.push_and_pull(self.history[1], h, batch_size, mfgs[1].srcdata['_ID'].cpu())
        h_dst = h[:mfgs[1].num_dst_nodes()]  # <---
        h = self.conv2(mfgs[1], (h, h_dst))  # <---
        return h
    
    # @torch.no_grad()
    # def forward_once(self, mfgs, x, batch_size):
    #     with tqdm.tqdm(train_dataloader) as tq:
    #         for step, (input_nodes, output_nodes, mfgs) in enumerate(tq):
    #             h_dst = x[:mfgs[0].num_dst_nodes()]  # <---
    #             # print("h_dst.shape", h_dst.shape)
    #             h = self.conv1(mfgs[0], (x, h_dst))  # <---
    #             h = F.relu(h)




model = Model(in_feats=lm.__num_node_features__, h_feats=64, num_classes=num_classes).to(device)
# model = Model(in_feats=1433, h_feats=64, num_classes=7).to(device)


In [ ]:
# opt = torch.optim.Adam(list(model.parameters())+list(lm.parameters())) # 
opt = torch.optim.Adam([
    {'params': lm.parameters(), 'lr': 1e-4},
    {'params': model.parameters(), 'lr': 0.01}])
valid_dataloader = dgl.dataloading.DataLoader(
    graph, val_nids, sampler,
    batch_size=64,
    shuffle=False,
    drop_last=False,
    num_workers=0,
    device=device
)

In [ ]:
import tqdm
import sklearn.metrics

best_accuracy = 0
best_model_path = 'model.pt'





for epoch in range(100):
    model.train()

    with tqdm.tqdm(train_dataloader) as tq:
        for step, (input_nodes, output_nodes, mfgs) in enumerate(tq):
            if epoch == 0:
                inputs = [text[i] for i in input_nodes]
                with torch.no_grad():
                    x = lm(inputs)
                    model.forward_once(mfgs, x)
                continue

                

            # print(111111)
            # print(mfgs[0].srcdata)
            # inputs = mfgs[0].srcdata['x']
            # break
            # print(inputs.shape, input_nodes.shape)
            inputs = [text[i] for i in output_nodes]
            labels = mfgs[-1].dstdata['y']
            
            inputs = lm(inputs).to(device)
            # mfgs = mfgs.to(device)
            # print(inputs.shape, labels.shape)
            # print(len(labels), inputs.shape, input_nodes.shape, output_nodes.shape)    
            # break        
            # predictions = model(mfgs, inputs, )
            # print(inputs.device, model.device)
            predictions = model(mfgs=mfgs, x=inputs, batch_size=16)
            labels = torch.flatten(labels)
            # print(predictions.device, labels.device)
            loss = F.cross_entropy(predictions, labels)
            # loss = torch.tensor(0.)

            opt.zero_grad()
            loss.backward()
            opt.step()

            accuracy = sklearn.metrics.accuracy_score(labels.cpu().numpy(), predictions.argmax(1).detach().cpu().numpy())

            tq.set_postfix({'loss': '%.03f' % loss.item(), 'acc': '%.03f' % accuracy}, refresh=False)

            del input_nodes, output_nodes, mfgs, inputs, labels, predictions, loss
            torch.cuda.empty_cache()
            # print(torch.cuda.mem_get_info())
    model.eval()

    predictions = []
    labels = []
    with torch.no_grad() and tqdm.tqdm(valid_dataloader) as tq, torch.no_grad():
        for input_nodes, output_nodes, mfgs in tq:
            # inputs = [text[i] for i in input_nodes]
            # print(type(mfgs[0]))
            inputs = mfgs[0].srcdata['x']
            labels.append(mfgs[-1].dstdata['y'].cpu().numpy())
            # inputs = lm(inputs).to(device)
            predictions.append(model(mfgs, inputs).argmax(1).cpu().numpy())
        predictions = np.concatenate(predictions)
        # print(predictions)
        labels = np.concatenate(labels)
        accuracy = sklearn.metrics.accuracy_score(labels, predictions)
        # print('Epoch {} Validation Accuracy {}'.format(epoch, accuracy))
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), best_model_path)
        #     inputs = mfgs[0].srcdata['feat']
        #     labels.append(mfgs[-1].dstdata['label'].cpu().numpy())
        #     predictions.append(model(mfgs, inputs).argmax(1).cpu().numpy())
        # predictions = np.concatenate(predictions)
        # labels = np.concatenate(labels)
        # accuracy = sklearn.metrics.accuracy_score(labels, predictions)
        print('Epoch {} Validation Accuracy {} Best Accuracy {}'.format(epoch, accuracy, best_accuracy))
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), best_model_path)

        # Note that this tutorial do not train the whole model to the end.
        # break